IMPORT LIBRARY

In [1]:
# === Basic ===
import os
import glob
import warnings
warnings.filterwarnings("ignore")

# === Data handling ===
import pandas as pd
import numpy as np

# === Visualization (buat inspeksi) ===
import matplotlib.pyplot as plt
import seaborn as sns

# === Date handling ===
from datetime import datetime

# === Sklearn utilities (nanti kepakai) ===
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
ISPU_DIR = "ISPU"
CUACA_DIR = "cuaca-harian"
JP_DIR = "jumlah-penduduk"
KAS_DIR = "kualitas-air-sungai"
LN_DIR = "libur-nasional"
NDVI_DIR = "NDVI"

ISPU = os.listdir(ISPU_DIR)
cuaca_harian = os.listdir(CUACA_DIR)
jumlah_penduduk = os.listdir(JP_DIR)
kualitas_air_sungai = os.listdir(KAS_DIR)
libur_nasional = os.listdir(LN_DIR)
ndvi = os.listdir(NDVI_DIR)

In [6]:
from collections import defaultdict

def audit_ispu_columns(ispu_dir):
    column_map = defaultdict(list)

    for file in os.listdir(ispu_dir):
        path = os.path.join(ispu_dir, file)
        df = pd.read_csv(path, nrows=5)

        for col in df.columns:
            key = col.lower().strip()
            column_map[key].append(file)

    return column_map


col_audit = audit_ispu_columns(ISPU_DIR)

for col, files in col_audit.items():
    print(f"\nKolom: {col}")
    for f in files:
        print("  -", f)



Kolom: periode_data
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2018-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2019-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2015-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2021-komponen-data.csv
  - data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-2023-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2010-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2014-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2020-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2011-komponen-data.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2017-komponen-data.csv
  - data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-komponen-data-2024.csv
  - indeks-standar-pencemaran-udara-(ispu)-tahun-2012-komponen-data.csv
  - data-indeks-standar-pencemar-udara-(ispu)-di-provinsi-dki-jakarta-kom

In [ ]:
COLUMN_MAPPING = {
    # ===== Particulate =====
    "pm10": "pm10",
    "pm_10": "pm10",
    "pm_sepuluh": "pm10",

    "pm25": "pm25",
    "pm_duakomalima": "pm25",

    # ===== Gas =====
    "so2": "so2",
    "sulfur_dioksida": "so2",

    "co": "co",
    "karbon_monoksida": "co",

    "o3": "o3",
    "ozon": "o3",

    "no2": "no2",
    "nitrogen_dioksida": "no2",

    # ===== Critical parameter =====
    "critical": "critical",
    "parameter_pencemar_kritis": "critical",

    # ===== Kategori =====
    "categori": "categori",
    "kategori": "categori",

    # ===== Stasiun =====
    "lokasi_spku": "stasiun",
    "stasiun": "stasiun",
}

def rename_columns_safely(df):
    df.columns = [c.lower().strip() for c in df.columns]
    rename_dict = {c: REAL_COLUMN_MAPPING[c] 
                for c in df.columns if c in REAL_COLUMN_MAPPING}
    return df.rename(columns=rename_dict)


STATION_CANDIDATES = [
    "stasiun",
    "lokasi_spku",
    "lokasi",
    "nama_lokasi"
]

def detect_stasiun_column(df):
    for col in df.columns:
        if col.lower() in STATION_CANDIDATES:
            return col
    return None

def normalize_kategori(x):
    x = str(x).strip().lower()

    if "tidak ada data" in x:
        return np.nan

    if "baik" in x:
        return "Baik"
    if "sedang" in x:
        return "Sedang"
    if "sangat" in x:
        return "Sangat Tidak Sehat"
    if "tidak sehat" in x:
        return "Tidak Sehat"
    if "berbahaya" in x:
        return "Berbahaya"

    return np.nan

import re

def normalize_stasiun(x):
    x = str(x).upper()

    match = re.search(r'DKI[1-5]', x)
    if match:
        return match.group(0)

    return np.nan

import pandas as pd
import numpy as np
import re

def excel_serial_to_datetime(x):
    try:
        return pd.to_datetime("1899-12-30") + pd.to_timedelta(float(x), unit="D")
    except:
        return pd.NaT


def smart_parse_tanggal(df, filename):

    if "tanggal" not in df.columns:
        return df

    # ===== CASE 1: Excel serial number =====
    is_excel_serial = df["tanggal"].astype(str).str.match(r'^\d+(\.\d+)?$')
    if is_excel_serial.any():
        df.loc[is_excel_serial, "tanggal"] = df.loc[
            is_excel_serial, "tanggal"
        ].apply(excel_serial_to_datetime)

    # ===== CASE 2: Sudah format string tanggal normal =====
    parsed = pd.to_datetime(df["tanggal"], errors="coerce", dayfirst=True)

    # coba alternatif kalau banyak gagal
    if parsed.isna().mean() > 0.3:
        parsed = pd.to_datetime(df["tanggal"], errors="coerce", dayfirst=False)

    df["tanggal"] = parsed

    # ===== CASE 3: Tanggal TANPA tahun (year jadi NaN) =====
    missing_year = df["tanggal"].dt.year.isna()

    if missing_year.any():
        df.loc[missing_year, "tanggal"] = df.loc[missing_year, "tanggal"].apply(
            lambda x: x.replace(year=year_from_file) if pd.notna(x) else x
        )

    return df


def standardize_ispu(df):
    df.columns = [c.lower().strip() for c in df.columns]

    # rename semua kolom yg maknanya sama
    df = df.rename(columns=COLUMN_MAPPING)

    # deteksi kolom stasiun kalau belum ada
    if "stasiun" not in df.columns:
        st_col = detect_stasiun_column(df)
        if st_col:
            df["stasiun"] = df[st_col]

    # tanggal
    if "tanggal" in df.columns:
        df["tanggal"] = pd.to_datetime(df["tanggal"], errors="coerce")

    return df



looping penggabungan

In [ ]:
# ispu_frames = []

# for file in os.listdir(ISPU_DIR):
#     path = os.path.join(ISPU_DIR, file)
#     df = pd.read_csv(path)

#     df_clean = standardize_ispu(df)
#     ispu_frames.append(df_clean)

# ispu_clean = pd.concat(ispu_frames, ignore_index=True)

def check_columns_after_standardization(ISPU_DIR):
    for file in os.listdir(ISPU_DIR):
        path = os.path.join(ISPU_DIR, file)
        df = pd.read_csv(path)
        df = standardize_ispu(df)

        print(f"\n==== {file} ====")
        print(sorted(df.columns))

